## Deploy Clip model on SageMaker

---
This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.
![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

---

In this notebook, we will deploy a [CLIP model](https://huggingface.co/openai/clip-vit-base-patch32) on the SageMaker endpoint with DJLServing container image.

### Setup

In [1]:
!pip install sagemaker boto3 huggingface_hub --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.55 requires botocore==1.34.55, but you have botocore 1.34.70 which is incompatible.


In [1]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
import json
import base64

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

In [3]:
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = "clip"  # folder within bucket where code artifact will go
s3_model_prefix = "model_clip"  # folder within bucket where code artifact will go
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{model_bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

Pretrained model will be uploaded to ---- > s3://sagemaker-us-west-2-513489159680/model_clip/


## Prepare inference script and container image

In [4]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=sess.boto_session.region_name, version="0.22.1"
)
inference_image_uri

'763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118'

In this notebook, we will store the model artifacts on S3 and load the model directly from S3

The [Large Model Inference (LMI)](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html) container uses [s5cmd](https://github.com/peak/s5cmd) to download data from S3 which significantly reduces the speed when loading model during deployment. Therefore, we recommend to load the model from S3 by following the below section to download the model from Hugging Face and upload the model on S3. Note that, if you choose to load the model directly from Hugging Face during model deployment, you can prepare the model tarbal file and upload to S3 without downloading the model locally.

### Download the model from Hugging Face and upload the model artifacts on Amazon S3
If you intend to download your copy of the model and upload it to a s3 location in your AWS account, please follow the below steps, else you can skip to the next step.

In [5]:
from huggingface_hub import snapshot_download
from pathlib import Path


# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path("./clip-model")
local_model_path.mkdir(exist_ok=True)
clip_model_name = "openai/clip-vit-base-patch32"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=clip_model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Please make sure the file is downloaded correctly by checking the files exist in the newly created folder `blip2-model/models--Salesforce--<model-name>/snapshots/...` before running the below cell.

In [6]:
# upload the model artifacts to s3
model_artifact = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.s3url={model_artifact}")

Model uploaded to --- > s3://sagemaker-us-west-2-513489159680/model_clip
We will set option.s3url=s3://sagemaker-us-west-2-513489159680/model_clip


In [7]:
!rm -rf {local_model_path}

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or post-processing of the model's predictions.

However, in this notebook, we demonstrate how to deploy a model with custom inference code.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties`, `model.py`, and `requirements.txt`.
- `serving.properties` is the configuration file that can be used to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration. For more details on the configuration options and an exhaustive list, you can refer the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html).
- `model.py` is the script handles any requests for serving.
- `requirements.txt` is the text file containing any additional pip wheel need to install. 

If you want to download the model from huggingface.co, you can set option.model_id. The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co. If you set the model_id to a s3 url, the DJL will download the model artifacts from s3 and swap the model_id to the actual location of the model artifacts. In your script, you can point to this value to load the pre-trained model.
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which the model needs to be partitioned. This parameter also controls the number of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine, and we are creating 8 partitions then we will have 1 worker per model to serve the requests.


In [8]:
%%writefile clip/serving.properties
engine = Python
option.tensor_parallel_degree = 1
option.model_id = {{s3url}}

Overwriting clip/serving.properties


In [9]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("clip/serving.properties").open().read())
Path("clip/serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
!pygmentize clip/serving.properties | cat -n

     1	engine = Python
     2	option.tensor_parallel_degree = 1
     3	option.model_id = s3://sagemaker-us-west-2-513489159680/model_clip/


## Prepare the model tarball file and upload to S3

In [21]:
%%sh
tar czvf model.tar.gz clip/

clip/
clip/.ipynb_checkpoints/
clip/.ipynb_checkpoints/model-checkpoint.py
clip/.ipynb_checkpoints/serving-checkpoint.properties
clip/model.py
clip/requirements.txt
clip/serving.properties


In [22]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-513489159680/clip/model.tar.gz


## Deploy model

In [23]:
from sagemaker.model import Model
from sagemaker.utils import name_from_base

model_name = name_from_base(clip_model_name.split("/")[-1])
model = Model(
    image_uri=inference_image_uri,
    model_data=s3_code_artifact,
    role=role,
    name=model_name,
)

In [24]:
%%time
endpoint_name = "text-embedding-" + model_name
model.deploy(initial_instance_count=1, instance_type="ml.g5.xlarge", endpoint_name=endpoint_name)

----------!CPU times: user 131 ms, sys: 2.62 ms, total: 134 ms
Wall time: 5min 32s


## Test Inference Endpoint

In [25]:
from PIL import Image
import base64
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")
endpoint_name = model.endpoint_name

In [26]:
def run_inference(endpoint_name, inputs):
    response = smr_client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(inputs))
    return response["Body"].read().decode("utf-8")

Let's ask our model to classify the objects appearing in the above image

In [33]:
prompts = "human"
inputs = {"prompt": prompts}
output = run_inference(endpoint_name, inputs)
output = json.loads(output)
print('output:',output['text_embedding'][0])

output: [0.02602003514766693, 0.05492720752954483, 0.02381453663110733, 0.2013988196849823, 0.020438581705093384, -0.10386614501476288, -0.10984423756599426, -0.8868318200111389, -0.11011698842048645, 0.20290571451187134, -0.2860334515571594, -0.1882266104221344, 0.2069985717535019, -0.05166694521903992, 0.3640192151069641, 0.45452529191970825, 0.38282567262649536, 0.3871825039386749, -0.03476647287607193, -0.21106913685798645, 0.41077306866645813, 0.2678365707397461, -0.2667296528816223, 0.10029122233390808, -0.15514914691448212, 0.11415030807256699, 0.33352363109588623, 0.2813056707382202, -0.28187039494514465, 0.1414727419614792, 0.14228418469429016, -0.019323375076055527, -0.27621400356292725, 0.05400625988841057, -0.1268874853849411, 0.13310043513774872, 0.358772337436676, 0.04202117398381233, -0.029715731739997864, -0.06073596328496933, -0.363821417093277, 0.10478681325912476, 0.11943098902702332, 0.4621464014053345, 0.04568805545568466, 0.3895756006240845, -0.07266345620155334, 

## Clean up
Uncomment the below cell to delete the endpoint and model when you finish the experiment

In [ ]:
sm_client.delete_model(ModelName=model_name)
sm_client.delete_endpoint(EndpointName=endpoint_name)

## Notebook CI Test Results
This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|llm-workshop|lab13-clip-interrogator|deploy-clip-model-on-sagemaker.ipynb)